In [1]:
import os
from glob import glob
import pathlib
import shutil

import earthpy.appeears as etapp
import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import requests
import rioxarray as rxr
import rioxarray.merge as rxrm
import xarray as xr
import xrspatial

data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
project_dir = os.path.join(data_dir, 'grassland-analysis')
#ndvi_dir = os.path.join(data_dir, 'oakland-green-space', 'processed')

for a_dir in [data_dir, project_dir]:
    if not os.path.exists(a_dir):
        os.makedirs(a_dir)

### Habitat Suitability Modeling for Sorghastrum nutans

We are building a habitat suitability model for this grassland type. Research the grass and add information and citations here!

I am going to analyze the Pawnee National Grassland and Thunder Basin National Grassland units. This data is downloaded from here, make sure to provide a citation!

https://www.gbif.org/species/2704414

In [2]:
# Download grassland unit shapefile
gl_unit_path = os.path.join(
    data_dir,
    'earthpy-downloads',
    'S_USA.NationalGrassland',
    'S_USA.NationalGrassland.shp'
)
if not os.path.exists(gl_unit_path):
    print('downloading ' + gl_url)
    gl_url = ('https://data.fs.usda.gov/geodata/edw/'
              'edw_resources/shp/S_USA.NationalGrassland.zip')
    gl_zip = et.data.get_data(url=gl_url)
    
gl_unit_gdf = (
    gpd.read_file(gl_unit_path).set_index('GRASSLANDN')
    .loc[['Thunder Basin National Grassland', 'Pawnee National Grassland']]
    .to_crs(4326)
)
gl_unit_gdf

NATIONALGR   GIS_ACRES  SHAPE_AREA  \
GRASSLANDN                                                               
Thunder Basin National Grassland  295513010328  626249.208    0.282888   
Pawnee National Grassland         295523010328  208424.885    0.089972   

                                  SHAPE_LEN  \
GRASSLANDN                                    
Thunder Basin National Grassland  44.088050   
Pawnee National Grassland         15.341594   

                                                                           geometry  
GRASSLANDN                                                                           
Thunder Basin National Grassland  MULTIPOLYGON (((-105.46005 43.31908, -105.4601...  
Pawnee National Grassland         MULTIPOLYGON (((-104.58106 40.82664, -104.5810...

In [3]:
# Create site maps for each GL unit
gl_layout = hv.Layout()

for name, details in gl_unit_gdf.iterrows():
    gl_plot = gl_unit_gdf.loc[[name]].hvplot(
        geo=True, 
        tiles='EsriNatGeo',
        aspect='equal',
        width=800,
        height=600,
        line_color='black',
        fill_alpha=0,
        title='Site Map for ' + name
    )
    gl_layout+=gl_plot
gl_layout.cols(1)

# gl_plot = gl_unit_gdf.hvplot(
#     geo=True, 
#     tiles='EsriNatGeo',
#     aspect='equal',
#     width=800,
#     height=600,
#     line_color='black',
#     fill_alpha=0,
#     title='Site Map for Grassland Units'
# )
# gl_plot

:Layout
   .Overlay.I  :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]
   .Overlay.II :Overlay
      .WMTS.I     :WMTS   [Longitude,Latitude]
      .Polygons.I :Polygons   [Longitude,Latitude]

### Habitat Characteristics

We are going to use the following characteristics for the habitat model:

https://www.nrcs.usda.gov/plantmaterials/etpmcpg13196.pdf

- median silt soil percentage from 15cm to 30cm- http://hydrology.cee.duke.edu/POLARIS/PROPERTIES/v1.0/silt/p50/15_30/


In [4]:
# Download habitat characteristic data

# Identify bounds of units to be able to select right file to download
for unit, details in gl_unit_gdf.iterrows():
    bbox = etee.BBox(*details.geometry.bounds)
    print(unit + ' ' +str(details.geometry.bounds))

soil_list = ['lat4344_lon-105-104.tif',
             'lat4344_lon-106-105.tif',
             'lat4445_lon-105-104.tif',
             'lat4445_lon-106-105.tif',
             'lat4041_lon-104-103.tif',
             'lat4041_lon-105-104.tif']

# Download soils data per the specified list above
for file in soil_list:
    file_path = os.path.join(
        data_dir,
        'earthpy-downloads',
        file
    )
    if not os.path.exists(file_path):
        print("Downloading " + file)
        url = ('http://hydrology.cee.duke.edu/POLARIS/'
        'PROPERTIES/v1.0/silt/p50/15_30/' + file
              )
        print(url)
        et.data.get_data(url=url)
    else:
        print(file + " already downloaded.")

# Merge and clip soils rasters
tif_paths = glob(os.path.join(
    data_dir,
    'earthpy-downloads',
    '*.tif')
)
# print(tif_paths)
print("attempting to merge soils rasters...")
das = [rxr.open_rasterio(tp, masked=True).squeeze() for tp in tif_paths]
merged_soils = rxrm.merge_arrays(das).rio.clip_box(*gl_unit_gdf.total_bounds)

merged_soils.hvplot(rasterize=True, x='x', y='y', aspect='equal')
    

Thunder Basin National Grassland (-105.68534577740812, 43.13179205151148, -104.3147230581148, 44.78726284154685)
Pawnee National Grassland (-104.79144253125483, 40.609566404744555, -103.57328571411065, 41.001847062442295)
lat4344_lon-105-104.tif already downloaded.
lat4344_lon-106-105.tif already downloaded.
lat4445_lon-105-104.tif already downloaded.
lat4445_lon-106-105.tif already downloaded.
lat4041_lon-104-103.tif already downloaded.
lat4041_lon-105-104.tif already downloaded.
attempting to merge soils rasters...


:DynamicMap   []
   :Image   [x,y]   (value)

In [11]:
type(das)

list

In [7]:
# Isolate soil areas for analysis

# Select sand values from array that exceed threshold; 1=possible indian grass, 0=not possible
# Adjust values based on your soil criteria per this diagram https://en.wikipedia.org/wiki/Loam#/media/File:SoilTexture_USDA.svg
silt_optimal_indiangrass = ((merged_soils >= 0) & (merged_soils <= 50))
silt_optimal_indiangrass = silt_optimal_indiangrass.astype(int)
silt_plot = silt_optimal_indiangrass.hvplot(rasterize=True, x='x', y='y', aspect='equal')
silt_plot

:DynamicMap   []
   :Image   [x,y]   (value)

In [48]:
type(silt_optimal_indiangrass)
silt_optimal_indiangrass
silt_optimal_indiangrass.values

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [16]:
# Download SRTM elevation data, save multiple arrays to a dictionary

elev_dict = {}
aspect_dict = {}
aspect_list =[]
elev_path_list = []
for name, details in gl_unit_gdf.iterrows():
    print("Attempting to download SRTM data for " + name)
    download_key = name.replace(" ", "-")
    
    srtm_dir = os.path.join(
        data_dir,
        'earthpy-downloads',
        'Final-Project',
        'SRTM',
    )
    elev_path_list.append(srtm_dir + "/" + download_key)
    print (elev_path_list)                      
    dl_gdf = (
        gl_unit_gdf
        .loc[[name]]
    )

    # Initialize AppeearsDownloader for MODIS NDVI data
    srtm_downloader = etapp.AppeearsDownloader(
        download_key= download_key,
        ea_dir=srtm_dir,
        product='SRTMGL1_NC.003',
        layer='SRTMGL1_DEM',
        start_date="02-11-2000",
        end_date="02-21-2000",
        polygon=dl_gdf
    )
    # Download files if the download directory does not exist
    if not os.path.exists(srtm_downloader.data_dir):
        srtm_downloader.submit_task_request()
        print("Submitting download request for " + name)
        srtm_downloader.download_files()
    else:
        print("Data for " + name + " is already downloaded.")
    
    tif_paths = glob(
        os.path.join(
        srtm_downloader.data_dir,
            'SRTMGL1_NC.003*',
            '*.tif'
        )
    )
    
    elev_da = [rxr.open_rasterio(srtm_path, masked=True).squeeze() for srtm_path in tif_paths][0]
    elev_plot = elev_da.hvplot(    
        rasterize=True,
        x='x',
        y='y',
        aspect='equal',
        width=400,
        height=400
    )
    aspect_list.append(xrspatial.aspect(elev_da))
    # aspect_plot = xrspatial.aspect(elev_da).hvplot(
    #     rasterize=True,
    #     x='x',
    #     y='y',
    #     aspect='equal',
    #     width=400,
    #     height=400
    # )
aspect_merged = rxrm.merge_arrays(aspect_list).rio.clip_box(*gl_unit_gdf.total_bounds)
aspect_merged.hvplot(rasterize=True, x='x', y='y', aspect='equal')

# xrspatial.aspect(elev_da).hvplot(
#     rasterize=True,
#     x='x',
#     y='y',
#     aspect='equal'
# )
# xrspatial.aspect(elev_da).hvplot(
#     rasterize=True,
#     x='x',
#     y='y',
#     aspect='equal'
# )


Attempting to download SRTM data for Thunder Basin National Grassland
['C:\\Users\\Pete\\earth-analytics\\data\\earthpy-downloads\\Final-Project\\SRTM/Thunder-Basin-National-Grassland']
Data for Thunder Basin National Grassland is already downloaded.
Attempting to download SRTM data for Pawnee National Grassland
['C:\\Users\\Pete\\earth-analytics\\data\\earthpy-downloads\\Final-Project\\SRTM/Thunder-Basin-National-Grassland', 'C:\\Users\\Pete\\earth-analytics\\data\\earthpy-downloads\\Final-Project\\SRTM/Pawnee-National-Grassland']
Data for Pawnee National Grassland is already downloaded.


:DynamicMap   []
   :Image   [x,y]   (aspect)

In [20]:
# Prepare aspect data for model
# Southeast/south/southwest values are from 112.5 to 247.5
# Reference for aspect values can be found here https://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-aspect-works.htm#ESRI_SECTION1_4198691F8852475A9F4BC71246579FAA

aspect_optimal_indiangrass = ((aspect_merged >= 112.5) & (aspect_merged <= 247.5))
# aspect_optimal_indiangrass = aspect_merged.astype(int)
aspect_optimal_plot = aspect_optimal_indiangrass.hvplot(rasterize=True, x='x', y='y', aspect='equal')
aspect_optimal_plot


:DynamicMap   []
   :Image   [x,y]   (aspect)

In [17]:
# Download MACA climate model data

maca_url = (
    'http://thredds.northwestknowledge.net:8080/thredds/ncss'
    '/agg_macav2metdata_pr_CCSM4_r6i1p1_historical_1950_2005_CONUS_monthly.nc'
    '?var=precipitation'
    '&disableLLSubset=on'
    '&disableProjSubset=on'
    '&horizStride=1'
    '&time_start=1980-01-15T00%3A00%3A00Z'
    '&time_end=1980-12-15T00%3A00%3A00Z'
    '&timeStride=1'
    '&accept=netcdf'
)
maca_response = requests.get(maca_url)
print(maca_response)
with open('maca.nc', 'wb') as maca_file:
    maca_file.write(maca_response.content)
maca_ds = xr.open_dataset('maca.nc')
maca_ds = maca_ds.assign_coords(lon= maca_ds.lon-360)
precip_da = maca_ds.precipitation
precip_da.rio.write_crs("epsg:4326", inplace=True)
precip_da.rio.set_spatial_dims('lon', 'lat', inplace=True)

<Response [200]>


<xarray.DataArray 'precipitation' (time: 12, lat: 585, lon: 1386)>
[9729720 values with dtype=float32]
Coordinates:
  * time     (time) object 1980-01-15 00:00:00 ... 1980-12-15 00:00:00
  * lat      (lat) float64 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.15 -67.11 -67.06
    crs      int32 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    grid_mapping:   crs
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [28]:
precip_clip = precip_da.rio.clip_box(*gl_unit_gdf.total_bounds).mean('time')
precip_plot = precip_clip.hvplot()*gl_unit_gdf.hvplot(aspect='equal')
    
precip_plot


:Overlay
   .Image.I    :Image   [lon,lat]   (precipitation)
   .Polygons.I :Polygons   [x,y]

In [29]:
# Isolate precip areas for analysis, 28 to 114 cm


# Select sand values from array that exceed threshold; 1=possible indian grass, 0=not possible
# need values to insert below, look here https://en.wikipedia.org/wiki/Loam#/media/File:SoilTexture_USDA.svg
precip_optimal_indiangrass = ((precip_clip >= 28) & (precip_clip <= 114))
precip_optimal_plot = precip_optimal_indiangrass.hvplot(rasterize=True, x='lon', y='lat', aspect='equal')
precip_optimal_plot

:DynamicMap   []
   :Image   [lon,lat]   (precipitation)

In [26]:
precip_optimal_indiangrass.dims

('time', 'lat', 'lon')